In [ ]:
import os, time,kfp,json
from dkube.sdk import *

In [ ]:
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
username =  os.getenv("USERNAME")
dkube_url = os.getenv("DKUBE_URL")
api = DkubeApi(URL=dkube_url,token=token)

In [ ]:
code_name = generate("dkube-examples")
code = DkubeCode(username, name=code_name)
code.update_git_details("https://github.com/oneconvergence/dkube-examples.git")
api.create_code(code)

In [ ]:
dataset_name = generate("mnist")
dataset = DkubeDataset(username, name=dataset_name)
dataset.update_git_details("https://dkube.s3.amazonaws.com/datasets/mnist.pkl.gz")
dataset.update_dataset_source(source='pub_url')
api.create_dataset(dataset)

In [ ]:
model_name = generate("mnist")
model = DkubeModel(username, name=model_name)
model.update_model_source(source='dvs')
api.create_model(model)

In [ ]:
storage_op = kfp.components.load_component_from_file("/mnt/dkube/pipeline/components/storage/component.yaml")

In [ ]:
@kfp.dsl.pipeline(name='StorageOp',description='StorageOp component')

def storageop_artifacts():
    with kfp.dsl.ExitHandler(exit_op=storage_op("reclaim", token, namespace="kubeflow",uid="{{workflow.uid}}")):
            input_volumes = json.dumps(["{{workflow.uid}}-project@program://"+str(code_name),"{{workflow.uid}}-dataset@dataset://"+str(dataset_name),"{{workflow.uid}}-model@model://" + str(model_name)])
            storage  = storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes)
            
            train = kfp.dsl.ContainerOp(
                name="container-op",
                image="docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3",
                command="bash", 
                arguments=["-c", "ls /dataset"],
                pvolumes={
                         "/project": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-project"),
                         "/dataset": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-dataset"),
                         "/model": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-model")
                         }).after(storage)
           
            
          


In [ ]:
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(storageop_artifacts,arguments={})
